### Log Reader  

In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

### Read the log file

In [2]:
import os
from langchain.tools import tool
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

@tool
def summarize_log() -> str:
    """Read and return summary of the first few lines from each .log file. DON'T get into details of them."""
    log_dir = './logs'  # Fixed typo: was 'loog_dir'
    db_path = "./log_db"
    all_logs = []
    
    # Check if log directory exists
    if not os.path.exists(log_dir):
        return "Log directory './logs' does not exist"
    
    # Process each log file
    for file in os.listdir(log_dir):
        if file.endswith('.log'):  # Only process .log files
            file_path = os.path.join(log_dir, file)  # Fixed variable name
            try:
                with open(file_path, 'r', encoding='utf-8') as f:  # Added encoding
                    lines = f.readlines()[:10]  # Read first 10 lines
                    for line in lines:
                        all_logs.append(f"{file}: {line.strip()}")
            except Exception as e:
                all_logs.append(f"{file}: Error reading file - {str(e)}")
    
    summary = "\n".join(all_logs) if all_logs else "No log files found"
    
    # Check if database already exists
    if not (os.path.exists(db_path) and os.path.isdir(db_path)):
        try:
            # Splitting
            splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
            chunks = splitter.create_documents([summary])
            
            # Embedding
            embedding = OllamaEmbeddings(model="nomic-embed-text")
            db = Chroma.from_documents(chunks, embedding, persist_directory=db_path)
            db.persist()
            summary += "\n\n[Embedding] Done and DB created"
        except Exception as e:
            summary += f"\n\n[Embedding] Error creating DB: {str(e)}"
    else:
        summary += "\n\n[Embedding] DB already exists"
    
    return summary

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Define tools
@tool
def add_number(a: int, b: int) -> int:
    """Add two numbers and return the result"""
    return int(a) + int(b)

@tool
def substract_number(a: int, b: int) -> int:
    """Subtract two numbers and return the result"""
    return int(a) - int(b)

@tool
def multiply_number(a: int, b: int) -> int:
    """Multiply two numbers and return the result"""
    return int(a) * int(b)

# Register tools
tools = [add_number, substract_number, multiply_number, summarize_log] 

# Create agent (use structured chat to support multi-input tools)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
tool_by_name = {tool.name: tool for tool in tools}
tool_by_name

In [ ]:
#query = "What is the sum of 20 and 40"
#query = "What are  errors in my log files? Summarize them for me."
query = "Hown WudfCoInstaller are in log file?"

results = agent.run(query)
print(results)